<a href="https://colab.research.google.com/github/19K41A0503/NNDL/blob/main/NNDL_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

In [3]:
data = pd.read_excel('/content/drive/MyDrive/Assignment3.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [4]:
X = []
Y = []
for i in range(24, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-24])
    Y.append(data['Load (kW)'].iloc[i])

In [5]:
data = pd.DataFrame({'Prev Day Load' : X, 'Pres Day Load' : Y})
data.head()

,Prev Day Load,Pres Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [6]:
Xmax = np.max(data['Prev Day Load'])
Xmin = np.min(data['Prev Day Load'])
Ymax = np.max(data['Pres Day Load'])
Ymin = np.min(data['Pres Day Load'])
data['Prev Day Load'] = (data['Prev Day Load'] - Xmin) / (Xmax - Xmin)
data['Pres Day Load'] = (data['Pres Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Prev Day Load,Pres Day Load
0,0.397877,0.284300
1,0.293800,0.255799
2,0.276454,0.244501
3,0.310234,0.239965
4,0.332801,0.268466


In [7]:
x = data['Prev Day Load']
y = data['Pres Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

In [8]:
#Initialization
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1

# Loop for epochs
for _ in range(epochs):
    
    for i in range(Xtrain.shape[0]):
        # Calculating expected m and c values
        tempM = m + v*vm
        tempC = c + v*vc
        
        # Calculating Grad M
        gradM = -1 * (Ytrain.iloc[i] - tempM * Xtrain.iloc[i] - tempC) * Xtrain.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (Ytrain.iloc[i] - tempM * Xtrain.iloc[i] - tempC)
        
        # Calculating updated values of vm and vc
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        # Updating m and c values
        m += vm
        c += vc

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.726053683812404
c = 0.12526803713967796


In [9]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

In [10]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

In [11]:
# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

# Testing Error Calculation
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 530.4613221719834
MSE : 517038.46279591543
RMSE : 719.0538664077368
Testing Error :
MAE : 543.3344444841334
MSE : 492586.59615103604
RMSE : 701.8451368721137


In [12]:
pd.DataFrame({'Actual Values' : trainY, 'Predicted Values' : train_pred})

,Actual Values,Predicted Values
0,4221.85392,4613.978731
1,7609.43808,6534.370279
2,5847.05880,5942.226916
3,4672.50300,5006.816959
4,7883.78688,6645.736691
...,...,...
1939,5830.37964,5699.801250
1940,6110.49600,5894.239763
1941,5289.94368,5463.260802
1942,7962.97392,7580.580761


In [13]:
pd.DataFrame({'Actual Values' : testY, 'Predicted Values' : test_pred})

,Actual Values,Predicted Values
0,6161.624640,5692.331552
1,5761.792440,6936.375865
2,7390.270800,6427.757311
3,4597.368840,5868.888059
4,5242.712040,5983.649789
...,...,...
211,6764.319072,7531.122304
212,7390.270800,6760.498421
213,6550.077600,6577.151279
214,5722.043040,5587.189888


In [15]:
x = float(input('Enter the load at present hour on previouus day : '))
x = (x - Xmin) / (Xmax - Xmin)
prediction = m * x + c
prediction = (prediction * (Ymax - Ymin)) + Ymin
print('Predicted load at present hour :', prediction)

Enter the load at present hour on previouus day : 2444
Predicted load at present hour : 3384.2770569251443
